In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np

In [2]:
model_path = 'pose_landmarker.task'

In [3]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,num_poses=6,min_pose_detection_confidence=0.3)
landmarker=PoseLandmarker.create_from_options(options)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1770349652.899111   21985 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1770349652.957401   21989 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
def avg(landmarks,m,per,l,p):
    return int((int(landmarks[l].x*m)+int(landmarks[p].x*m))/per)
def pointer(landmarks,image):
    color=(237,16,160)
    edit=image#use .copy so that it does not alter real image
    n,m,_=np.shape(edit)
    lld=[[16,14],[14,12],[11,13],[13,15],[28,26],[26,24],[23,25],[25,27]]
    rad=int((landmarks[7].x-landmarks[8].x)*m)    
    for p,q in lld:
        x1,y1=int(landmarks[p].x*m),int(landmarks[p].y*n)
        x2,y2=int(landmarks[q].x*m),int(landmarks[q].y*n)
        if (p==14):
            x2,y2=avg(landmarks,m,2,11,12),int(landmarks[12].y*n)
            cv2.line(edit,(int(landmarks[0].x*m),int(landmarks[0].y*n+rad)),(x2,y2),color,30,2)
        if (q==13):
            x1,y1=avg(landmarks,m,2,11,12),int(landmarks[12].y*n)
            cv2.line(edit,(avg(landmarks,m,2,23,24),int(landmarks[23].y*n)),(x1,y1),color,30,2)
        if (p==26):
            x2,y2=(avg(landmarks,m,2,23,24),int(landmarks[23].y*n))
        if (q==25):
            x1,y1=(avg(landmarks,m,2,23,24),int(landmarks[23].y*n))
        cv2.line(edit,(x1,y1),(x2,y2),color,30,2)

    cv2.circle(edit,(int(landmarks[0].x*m),int(landmarks[0].y*n)),rad,color,30,1)     
    return edit    

In [5]:
video=cv2.VideoCapture(0)
while (True):
    try:
        if video.isOpened():
            _,image=video.read()
            # image=cv2.resize(image,(1900,900))
            image=cv2.flip(image,1)
            back=(np.ones(image.shape)*0).astype(np.uint8)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            pose_landmarker_result = landmarker.detect(mp_image)
            edit=pointer(pose_landmarker_result.pose_landmarks[0],back)
            cv2.imshow("marks",edit)
            if cv2.waitKey(1) & 0xff==ord(" "):
                break
    except:
        continue        
cv2.destroyAllWindows()
video.release()

W0000 00:00:1770349653.440057   21982 landmark_projection_calculator.cc:78] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
QFontDatabase: Cannot find font directory /home/yogendra/Desktop/stickmen/python_39/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/Desktop/stickmen/python_39/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/yogendra/Desktop/stickmen/python_39/lib/python3.9/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /h